In [ ]:
!pip install --upgrade langchain-gigachat
!pip install --upgrade langgraph

In [40]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from langchain_gigachat import GigaChat
from langchain_gigachat.tools.giga_tool import giga_tool
from pydantic import BaseModel, Field
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent
import json, re
from langchain import LLMChain, PromptTemplate
from typing import List
import time
from langgraph.checkpoint.memory import MemorySaver
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

In [42]:
class GetBankSlugResult(BaseModel):
    """
    Модель результата работы функции, возвращающей slug банка.

    Attributes:
        slug (str): Уникальный символьный идентификатор (slug) банка на banki.ru,
                    необходимый для формирования URL при сборе отзывов.
    """
    slug: str = Field(description=(
            "Уникальный символьный идентификатор банка на banki.ru, "
            "который используется для построения конечного URL "
            "при парсинге отзывов."
        ))

In [43]:
few_shot_examples = [
    {
        "request": "Найди отзывы про сбербанк",
        "params": {"bank_name": "сбербанк"}
    },
    {
        "request": "Дай мне отзывы про альфабанк",
        "params": {"bank_name": "альфабанк"}
    },
    {
        "request": "Какой slug у сбербанка?",
        "params": {"bank_name": "сбербанк"}
    },
]

@giga_tool(few_shot_examples=few_shot_examples)
def get_bank_slug(bank_name: str = Field(description="Полное или приближённое название банка")) -> GetBankSlugResult:
    """
    Определяет slug банка на banki.ru по его названию.

    Выполняет HTTP-запрос к поисковому эндпоинту banki.ru,
    парсит названия и URL найденных банков. Если найден один
    вариант — возвращает его. Если найдено несколько — передаёт
    список вариантов LLM (GigaChat), чтобы оно выбрало самый
    подходящий slug.

    Args:
        bank_name (str): Название банка, введённое пользователем.

    Returns:
        GetBankSlugResult: Pydantic-модель с полем `slug`.
    """
    url = (f"https://www.banki.ru/banks/search/"
           f"?search[text]={bank_name}&search[type]=name")
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "html.parser")

    names = [a.get_text(strip=True)
             for a in soup.find_all("a", "widget__link")]
    slugs = [a["href"].split("/")[4]
             for a in soup.find_all("a", href=lambda h: h and "/services/responses/bank/" in h)]

    if not names:
        raise ValueError(f"«{bank_name}» не найдено на banki.ru")
    if len(names) == 1:
        return GetBankSlugResult(slug=slugs[0])

    options = "\n".join(f"{i+1}) {names[i]} — slug `{slugs[i]}`"
                        for i in range(len(names)))
    template = """
Пользователь запросил банк: "{bank_name}".
Найдены следующие варианты на banki.ru:
{options}

Пожалуйста, выбери наиболее подходящий вариант и ответь ровно одним словом — нужным slug из backticks.
    """.strip()
    prompt = PromptTemplate(
        input_variables=["bank_name", "options"],
        template=template
    )
    chain = LLMChain(llm=giga, prompt=prompt)
    response = chain.run(bank_name=bank_name, options=options).strip()
    print(f"! get_bank_slug: {bank_name}")
    return GetBankSlugResult(slug=response.strip("` \n"))

In [ ]:
class Review(BaseModel):
    """
    Описание одного отзыва о банке.
    """
    date: str = Field(description="Дата отзыва в формате YYYY-MM-DD")
    name: str = Field(description="Название отзыва")
    text: str = Field(description="Текст, написанный пользователем")
    rating: int = Field(description="Оценка банка (целое число от 1 до 5)")

class ParseReviewsResult(BaseModel):
    """
    Итог работы функции парсинга отзывов.
    """
    reviews: List[Review] = Field(description="Список отзывов, отфильтрованных по заданному периоду")

In [ ]:
few_shot_examples = [
        {
            "request": "Собери отзывы про Т-Банк за март 2025",
            "params": {
                "bank_name": "Т-Банк",
                "start": "2025-03-01",
                "end": "2025-03-31"
            }
        },
        {
            "request": "Собери отзывы про Сбербанк с 25 марта по 4 апреля 2025 года",
            "params": {
                "bank_name": "Сбербанк",
                "start": "2025-03-25",
                "end": "2025-04-04"
            }
        },
        {
            "request": "дай мне все отзывы про Т-Банк с 25 декабря 2024 года по 4 февраля 2025 года",
            "params": {
                "bank_name": "Т-Банк",
                "start": "2024-12-25",
                "end": "2025-02-04"
            }
        },
    ]

@giga_tool(few_shot_examples=few_shot_examples)
def parse_reviews(
    bank_name: str = Field(description="Полное или приближённое название банка"),
    start: str     = Field(description="Дата начала периода в формате YYYY-MM-DD"),
    end:   str     = Field(description="Дата окончания периода в формате YYYY-MM-DD"),
) -> ParseReviewsResult:
    """
    Парсит отзывы о банке с banki.ru за указанный период.

    Внутри:
    1. Получает slug через get_bank_slug.
    2. Проходит постранично по отзывам и собирает их в список.
    3. Останавливается, когда выходим за рамки end_date или когда страницы кончились.
    """
    slug_result = get_bank_slug.invoke({"bank_name": bank_name})
    slug = slug_result.slug
    
    start_date = datetime.strptime(start, "%Y-%m-%d")
    end_date   = datetime.strptime(end,   "%Y-%m-%d")
    collected = []
    page = 1

    while True:
        url = (
            f"https://www.banki.ru/services/responses/bank/"
            f"{slug}/?page={page}&is_countable=on"
        )
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, "html.parser")
        scripts = soup.find("script", attrs={"type": "application/ld+json"})
        json_scripts = json.loads(scripts.string, strict=False)
        items = json_scripts["review"]

        if not items:
            break

        for itm in items:
            date_str = itm["datePublished"][:10]
            dt = datetime.strptime(date_str, "%Y-%m-%d")
            if dt < start_date:
                return ParseReviewsResult(reviews=collected)
            if start_date <= dt <= end_date:
                raw_text = itm.get("reviewBody") or itm.get("description") or ""
                clean = BeautifulSoup(raw_text, "html.parser").get_text(" ", strip=True)
                text = re.sub(r"\s+", " ", clean)

                #text = itm["description"].replace("&lt;", "").replace("p&gt", "")
                name = itm["name"]
                raw_rating = itm.get("reviewRating", {}).get("ratingValue", "").strip()
                if not raw_rating or not re.fullmatch(r"\d+(\.\d+)?", raw_rating):
                    continue              # → пропускаем без рейтинга

                rating = int(round(float(raw_rating)))   # гарантированно int
                collected.append(Review(
                    date=dt.strftime('%Y-%m-%d'),
                    name=name,
                    text=text,
                    rating=rating
                ))
        page += 1
    print(f"! parse_reviews: {bank_name, start_date, end_date}")
    return ParseReviewsResult(reviews=collected)

In [ ]:
auth = "YOUR_KEY"

In [57]:
def chat(agent_executor, thread_id: str):
    config = {"configurable": {"thread_id": thread_id}}
    print("\033[93m" + f"Bot: Какие отзывы вы хотите найти? \nПример: 'Найди все отзывы про Сбербанк за 1-25 апреля 2025 года'" + "\033[0m")

    while True:
        user_input = input("Клиент: ")
        if user_input == "":
            break
        print(f"User: {user_input}")
        resp = agent_executor.invoke({"messages": [HumanMessage(content=user_input)]}, config=config)
        bot_answer = resp['messages'][-1].content
        print("\033[93m" + f"Bot: {bot_answer}" + "\033[0m")
        time.sleep(1)


system_msg = SystemMessage(content="""
Ты — помощник‑аналитик. Когда ты получаешь результат функции
parse_reviews, ОБЯЗАТЕЛЬНО выводи поле 'text' каждого отзыва целиком,
без сокращений и многоточий. НЕ указывай, кто автор отзыва!
ОБЯЗАТЕЛЬНО следуй нижеуказанному формату вывода отзывов!
Формат: <номер>. <date> · <name> · ★<rating>/5 <перенос строки><text>
""")

giga = GigaChat(
    credentials=auth,
    model='GigaChat',
    verify_ssl_certs=False,
    scope='GIGACHAT_API_PERS'
)

tools = [
    get_bank_slug,
    parse_reviews
]

giga_with_tools = giga.bind_functions(tools)

agent = create_react_agent(
    giga_with_tools,
    tools,
    checkpointer=MemorySaver(),
    prompt=system_msg
)

In [ ]:
chat(agent, "id_1")